<a href="https://colab.research.google.com/github/linbinbin/Langchain_practice/blob/main/cobol_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q langchain-chroma
!pip install -q tiktoken
!pip install -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## 準備：API キーの設定

In [ ]:

from google.colab import userdata
import openai
import os
# OPENAI_API_KEY を取得
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai.api_key
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = "source_RAG"
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"

In [ ]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

# Load
loader = GenericLoader.from_filesystem(
    ".",
    glob="**/*",
    suffixes=[".pli", ".mfs"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.COBOL, parser_threshold=500),
)
documents = loader.load()
len(documents)


2

In [ ]:
from langchain.schema import Document

class DelimiterTextSplitter:
    def __init__(self, delimiter="%PAGE;"):
        self.delimiter = delimiter

    def split(self, text):
        return text.split(self.delimiter)

    def split_documents(self, documents):
        split_documents = []
        for document in documents:
            split_texts = self.split(document.page_content)
            for split_text in split_texts:
                # Create a new Document object for each split text, retaining the original metadata
                split_documents.append(Document(page_content=split_text, metadata=document.metadata))
        return split_documents

# Initialize the custom splitter with the delimiter
splitter = DelimiterTextSplitter(delimiter="%PAGE;")

# Split the documents
split_texts = splitter.split_documents(documents)

# Process each split text (example processing: printing each split text)
#for i, text in enumerate(split_texts):
#    print(f"Split Text {i+1}:\n{text.strip()}\n")
len(split_texts)

116

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

cobol_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.COBOL, chunk_size=2000, chunk_overlap=200
)
texts = cobol_splitter.split_documents(documents)
len(texts)

372

## 文埋め込みと類似文書検索

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

db = Chroma.from_documents(split_texts, OpenAIEmbeddings(disallowed_special=()))
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
query = "bar.mfsは？"
retriever.invoke(query)

## Retrieval を活用したチャットモデルの制御

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.runnables import RunnableLambda
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_core.runnables.base import Runnable


llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)
document_chain = create_stuff_documents_chain(llm, prompt)

qa = create_retrieval_chain(retriever_chain, document_chain)

save_to_file_chain = RunnableLambda(lambda x: open(x["filename"], 'w').write(x["answer"]))

# 既存のチェーンに新しいチェーンを追加します。
qa = qa | save_to_file_chain



In [ ]:
question = """
１、bar.pliに書かれたプロシージャを順番に並べてください。
２、各プロシージャの処理内容を記載してください。
＃出力フォーマット
プログラム仕様書スタイルで、極力表、図の形式（マーメイド）で視認性を上げる

＃追加項目(以下順番で出力)
以下章分けして出力
２．１ 処理概要
2.２ このプログラムのInput/Output定義
２.3 処理フロー
"""
result = qa.invoke({"input": question, "filename": "bar.md"})
result["answer"]

In [ ]:
question = """
bar.mfsに書かれた表示のレイアウトをお答えください。
"""
result = qa.invoke({"input": question})
result["answer"]


In [ ]:
question = """
  M000_MAIN　に書かれた処理フローをお答えください。
"""
result = qa.invoke({"input": question, "filename": "bar.md"})
result

581

In [ ]:
from langchain_core.runnables import RunnableLambda
read_tool, write_tool, list_tool = FileManagementToolkit(selected_tools=["read_file", "write_file", "list_directory"],).get_tools()
save_to_file_chain = RunnableLambda(lambda x: write_tool(x["aa"], x["filename"]))
save_to_file_chain.invoke({"aa": "aaa", "filename": "bar.md"})
# A RunnableSequence constructed using the `|` operator
#sequence = RunnableLambda(lambda x: x + 1)
#sequence.invoke(1) # 4


AttributeError: 'str' object has no attribute 'parent_run_id'

In [ ]:
write_tool_lambda = lambda x: write_tool(x["content"], x["filename"])
write_tool_lambda({"content": "aaa", "filename": "bar.md"})

AttributeError: 'str' object has no attribute 'parent_run_id'

In [ ]:
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()
toolkit = FileManagementToolkit(
    #root_dir=str(working_directory.name)
    selected_tools=["read_file", "write_file", "list_directory"],
)  # If you don't provide a root_dir, operations will default to the current working directory
tools = toolkit.get_tools()
read_tool, write_tool, list_tool = tools
write_tool.invoke({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

### 準備：プロンプトテンプレートを作る

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="以下を参照して、質問に答えてください。\n\n{context}\n\n質問：{question}"
)
example = {"question":"これは質問です", "context":"これは外部知識です"}

prompt_template.invoke(example)

StringPromptValue(text='以下を参照して、質問に答えてください。\n\nこれは外部知識です\n\n質問：これは質問です')

### 準備：抽出器を作る

In [ ]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})
retriever.invoke("3月の売り上げは？")

### 準備：フォーマッターを作る
抽出器の出力を生テキストの形に成形する関数を用意しておく。

In [ ]:
def text_formatter(retriever_output):
  raw_text = retriever_output[0].page_content
  raw_text_wo_newline = raw_text.replace("\n", "")
  return raw_text_wo_newline

### 抽出・補完・生成のチェインを作る

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

retriever、prompt_template、llmが用意できたので、これらを組み合わせて、情報の流れを定義する。LangChainでは、これから書いていくような「インプットからアウトプットまでの流れ」をチェイン（chain）と呼ぶ。

In [ ]:

chain = ({"question":RunnablePassthrough(), "context":retriever|text_formatter}
         | prompt_template
         | llm
         )

In [ ]:
chain.invoke("3月の売上は？")

AIMessage(content='表に直接「3月の売上」が記載されていませんが、「当月まで累計実績」と「3ヶ月後累計推定」から逆算することができます。\n\n「当月まで累計実績」は61.172083、「3ヶ月後累計推定」は120.832083です。これから、3ヶ月分の売上を計算します：\n\\[ 3ヶ月分の売上 = 120.832083 - 61.172083 = 59.66 \\]\n\n次に、この3ヶ月分の売上を3で割ることで、1ヶ月あたりの売上の平均を求めます：\n\\[ 1ヶ月あたりの売上の平均 = 59.66 / 3 ≈ 19.887 \\]\n\nこれは近似値ですが、3ヶ月分の売上が3月、4月、5月のものと仮定すると、この平均値が3月の売上と推定できます。\n\nしたがって、3月の売上はおおよそ19.887と推定されます。', response_metadata={'token_usage': {'completion_tokens': 234, 'prompt_tokens': 230, 'total_tokens': 464}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8e56983-68e1-4cd9-9031-d0271448cff8-0', usage_metadata={'input_tokens': 230, 'output_tokens': 234, 'total_tokens': 464})

In [ ]:
resp=chain.invoke("月３、４、５の当月までの累計実績の変化を教えて")

In [ ]:
resp.content

'月3、4、5の当月までの累計実績の変化について、以下のようにまとめます。\n\n### 売上の累計実績\n- **月3**: 96.014928\n- **月4**: 100.049283\n- **月5**: 154.002083\n\n### 部門利益の累計実績\n- **月3**: 17.231823\n- **月4**: 23.017230\n- **月5**: 25.091665\n\n### 部門利益率の累計実績\n- **月3**: 0.172233\n- **月4**: 0.172233\n- **月5**: 0.162931\n\nこれらのデータから、次のような変化が見られます。\n\n#### 売上の累計実績の変化\n- 月3から月4への変化: 増加（96.014928 → 100.049283）\n- 月4から月5への変化: 大幅な増加（100.049283 → 154.002083）\n\n#### 部門利益の累計実績の変化\n- 月3から月4への変化: 増加（17.231823 → 23.017230）\n- 月4から月5への変化: 増加（23.017230 → 25.091665）\n\n#### 部門利益率の累計実績の変化\n- 月3から月4への変化: 変化なし（0.172233 → 0.172233）\n- 月4から月5への変化: 減少（0.172233 → 0.162931）\n\n以上のように、売上と部門利益は全体的に増加している一方で、部門利益率は月5で減少しています。'

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip freeze > request.txt